In [1]:
import numpy as np
import pandas as pd 
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
from ipywidgets import HTML
from ipyleaflet import Popup, Map, Marker, LayersControl
from IPython import display

class desafioBiodiversidade():
    def __init__(self,filePath):
        try:
            f = open(filePath,"r")
            self.originalData = f.readlines()
            self.data = list(map(lambda line: line.split(";"), self.originalData))
            f.close()
        except:
            print("Error: cannot open file.")
            data = None  
    
    #1
    #Para cada coluna identique a quantidade de linhas com dados faltantes (em alguns casos, 
    #o dado faltante é uma string vazia, em outros casos é uma string contendo algum valor do 
    #tipo: "sem informação"). 
    #Faça um método que retorna a média de dados faltantes por coluna
    def retornaMediaDadosFaltantesPorColuna(self):
        data = self.data
        nlines = len(data)-1
        
        # lista de tamanho igual ao numero de colunas
        nblank = [0]*len(data[0])
        
        # cada posicao da lista armazena o contador de linhas faltantes da coluna correspondente
        for line in data:
            for ind,column in enumerate(line):
                if column == "Sem Informações" or column == "":
                    nblank[ind]+=1
        
        # calcula a media dos dados faltantes de cada coluna
        nblank = [np.round((x/nlines)*100,2) for x in nblank]
        
        # passa a lista para um dicionario
        # para impressao utilizando DataFrame do Pandas
        dict = {'Titulo': data[0], 'Value': nblank}
        df = pd.DataFrame(dict)
        print(df.to_string(index=False))
        
        
    #2
    #Para cada item identifique até qual nível taxônomico a ocorrência foi identificada.
    #Seis campos definem a espécie: Filo;Classe;Ordem;Familia;Genero;Especie do mais genérico 
    #(Filo) para o mais específico (Especie), é comum que em alguns casos, o preenchimento pare em 
    #família ou genêro por exemplo.
    def retornaNivelTaxonomicoCadaOcorrencia(self):
        lista = []
        for i in self.originalData:
            lista.append(i.strip("\n").split(";"))

        filo = []
        for i in range(len(lista[0])):
            
            if lista[0][i] == 'Especie':
              
                for k in range(1,len(lista)):
                    
                    if lista[k][i] != 'Sem Informações':
                        filo.append('{} - Especie'.format(k))
                    
                    elif lista[k][i-1] != 'Sem Informações':
                        filo.append('{} - Genero'.format(k))
                        
                    elif lista[k][i-2] != 'Sem Informações':
                        filo.append('{} - Familia'.format(k))
                        
                    elif lista[k][i-3] != 'Sem Informações':
                        filo.append('{} - Ordem'.format(k))
                    
                    elif lista[k][i-4] != 'Sem Informações':
                        filo.append('{} - Classe'.format(k))
                        
                    elif lista[k][i-5] != 'Sem Informações':
                        filo.append('{} - Filo'.format(k))
                        
                    else:
                        filo.append('{} - Reino'.format(k))
     
        return filo  
    
    #3
    #Monte filtros de ocorrências por estados, nome de espécie (nome exato ou parte do nome) e 
    #categoria de ameaça, e outros filtros que julgar relevante.
    def retonaSeOcorrenciaExiste(self, filterAmeaca):
        self.AmeacaList = []
        self.plotMap = []
        self.dataAux = []
        self.dataT = []
        self.lines = [lines for lines in self.originalData]
        self.alldata = [dados.strip("\n").split(";") for dados in self.lines]
        # Lê cada coluna por vez        
        for j in range(len(self.alldata[0])):        
            for i in range(len(self.alldata)-1):
                self.dataAux.append(self.alldata[i][j])
                # Cria uma nova "matriz" transposta com os dados
            self.dataT.append(self.dataAux)
            self.dataAux = []
        
        
        ##### Filtros de busca
        # Filtro de ameaça
        for i in range(len(self.dataT)):
            if self.dataT[i][0] == "Estado de conservacao":
                for j in range(len(self.dataT[i])):
                    if self.dataT[i][j] == filterAmeaca :
                        self.plotMap.append([self.dataT[i+7][j],self.dataT[i+8][j], self.dataT[i+5][j]])
                        self.AmeacaList.append(["Amostra {}: {}: ".format(j, self.dataT[i-12][j]), filterAmeaca])
                    
        return self.AmeacaList
    
    def retornaLatLongitude(self):
        return self.plotMap
        
        
    #4
    #Crie uma funcionalidade para avaliar se a informação de longitude e latitude 
    #corresponde a informação presente na localização, para isso você pode utilizar uma 
    #biblioteca de consulta reversa de lat/log como por exemplo o 
    #https://opencagedata.com/tutorials/geocode-in-python
    def metodoPlotMap(self, vectorInformacoes):
        key = '0573d875005f45bebe452ff81c26b684'
        geocoder = OpenCageGeocode(key)

        i = 0
        center = (0.0, 0.0)
        itemMapa = Map(center = center, zoom = 3, close_popup_on_click = False)
        itemMapa.add_control(LayersControl())
        for item in vectorInformacoes:
            #if i <= 15:
                center = (item[0], item[1])
                marker = Marker(location = center, draggable = False)
                teste = HTML()
                teste.value = "<b>" + item[2] + "</b>"
                popup = Popup(
                    location = center,
                    child = teste,
                    close_button = True,
                    auto_close = False,
                    close_on_escape_key = False
                )
                itemMapa.add_layer(marker)
                itemMapa.add_layer(popup)
            #i+=1
        return itemMapa
        

#PARTE GABRIELA
filePath = "portalbio_export_16-10-2019-14-39-54.csv"
biodiversidade = desafioBiodiversidade(filePath)
biodiversidade.retornaMediaDadosFaltantesPorColuna()
#PARTE GABRIELA

#PARTE BRUNA
print(biodiversidade.retornaNivelTaxonomicoCadaOcorrencia())
#PARTE BRUNA

#PARTE RAFAEL
print(biodiversidade.retonaSeOcorrenciaExiste("Espécie Ameaçada"))
#PARTE RAFAEL

#PARTE RAFAEL + DANIEL
map = biodiversidade.metodoPlotMap(biodiversidade.retornaLatLongitude())
#PARTE RAFAEL + DANIEL
map

                              Titulo   Value
                 Nome da instituicao  100.00
                Sigla da instituicao    0.00
               Nome da base de dados    0.00
              Sigla da base de dados    0.00
           Responsavel pelo registro    0.08
        Numero do registro no portal    0.00
 Numero do registro na base de dados    6.01
                    Data do registro    3.50
                      Data do evento   54.45
                    Data de Carencia    0.00
                     Nome cientifico    1.69
                          Nome comum   74.21
    Nome cientifico na base de dados    0.00
                    Nivel taxonomico    1.69
                Numero de individuos   51.35
                               Reino  100.00
                                Filo    3.17
                              Classe    3.76
                               Ordem    4.29
                             Familia    4.73
                              Genero   16.51
          

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …